In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob
import cv2

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
# loading dataset

def load_data(path , split = 0.1):
    
    images = sorted(glob(os.path.join(path , "PNG/Original/*")))
    
    masks = sorted(glob(os.path.join(path , "PNG/Ground Truth/*")))
    
    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)
    
    # 612 total 
    # 61 test
    # 61 validate
    
    train_x , valid_x = train_test_split(images , test_size = valid_size, random_state = 42)
    
    train_y , valid_y = train_test_split(masks , test_size = valid_size, random_state = 42)
    
    
    train_x , test_x = train_test_split(train_x , test_size = test_size, random_state = 42)
    
    train_y , test_y = train_test_split(train_y , test_size = test_size, random_state = 42)
    
    return (train_x , train_y ) , (valid_x , valid_y) , (test_x , test_y)

In [4]:
def read_image(path):
    
    path = path.decode()
    x = cv2.imread(path , cv2.IMREAD_COLOR)
    x = cv2.resize(x , (256, 256))
    x = x/255.0
    # size is 256, 256 , 3
    return x


In [5]:
def read_mask(path):
    
    path = path.decode()
    x = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x , (256, 256))
    x = x/255.0
    # size is 256, 256
    x = np.expand_dims(x, axis = -1)
    # size is 256, 256 , 1
    
    return x

In [6]:
def tf_parse (x,y) :
    
    def _parse(x,y):
        
        x = read_image(x)
        y = read_mask(y)
        
        return x,y
    x,y = tf.numpy_function(_parse , [x,y] , [tf.float64 , tf.float64])
    
    x.set_shape([256,256,3])
    y.set_shape([256,256,1])
    
    return x,y

In [7]:
def tf_dataset(x,y,batch = 8):
    
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [8]:
if __name__ == "__main__":
    
    path = "/kaggle/input/cvcclinicdb"
    (train_x , train_y ) , (valid_x , valid_y) , (test_x , test_y) = load_data(path,0.1)
    
    # 612 total 
    # 61 test
    # 61 validate
    # 490 train
    
    ds = tf_dataset(test_x , test_y)
    
    for x, y in ds:
        
        print(x.shape , y.shape) # batch of 8 images with 8 img and 8 masks
        break

(8, 256, 256, 3) (8, 256, 256, 1)


In [9]:
# unet arch code

import tensorflow as tf
from tensorflow.keras.layers import Conv2D , BatchNormalization, Activation, Concatenate,UpSampling2D, Input, MaxPool2D
from tensorflow.keras.models import Model


def conv_block(x , num_filters) :
    
    x = Conv2D(num_filters , (3,3) , padding= "same" ) (x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters , (3,3) , padding= "same" ) (x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    return x


def build_model():
    
    size = 256
    
    num_filters= [16,32,64,128]
    
    inputs = Input(shape = (size,size,3))
    
    skip_x = []
    x = inputs
    
    for f in num_filters:
        x = conv_block(x,f)
        skip_x.append(x)
        x = MaxPool2D((2,2))(x)
    # bridge
    
    x = conv_block(x,num_filters[-1])
    
    num_filters.reverse()
    
    skip_x.reverse()
    
    for i , f in enumerate(num_filters):
        
        x = UpSampling2D((2,2))(x)
        xs = skip_x[i]
        x = Concatenate()([x,xs])
        x = conv_block(x,f)
        
    x = Conv2D(1,(1,1) , padding="same")(x)
    x = Activation("sigmoid") (x)
    
    return Model(inputs,x)

if __name__ == "__main__":
    
    model = build_model()
    model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [10]:

import tensorflow.keras.backend as K

from sklearn.metrics import jaccard_score,confusion_matrix

def IoU_coef(y_true, y_pred):
    
    print(y_true.shape)
    print(y_pred.shape)
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def dice_coef(y_true, y_pred):
    
    print(y_true.shape)
    print(y_pred.shape)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

In [11]:
from tensorflow import keras
import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf


In [12]:
# Cross Entropy
# Focal 
# Dice 
# Tversky
# Tversky Focal
# Combo
# Unified Focal (Sym)
# Unified Focal (Asym)


In [13]:
# dice_loss = DiceLoss()
# binary_focal_loss = BinaryFocalLoss()
# combo_loss = binary_crossentropy + dice_loss


In [14]:

def dice_coef(y_true, y_pred, smooth=100):        
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


In [15]:
def sym_unified_focal_loss(y_true, y_pred ):
    
    axis = identify_axis(y_true.get_shape())
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    def loss_function(y_true,y_pred):
        weight=0.5
        delta=0.6 
        gamma=0.5
        symmetric_ftl = symmetric_focal_tversky_loss(delta=delta, gamma=gamma)(y_true,y_pred)
        symmetric_fl = symmetric_focal_loss(delta=delta, gamma=gamma)(y_true,y_pred)
        if weight is not None:
            return (weight * symmetric_ftl) + ((1-weight) * symmetric_fl)
        else:
            return symmetric_ftl + symmetric_fl

    return loss_function(y_true,y_pred)

In [16]:
def iou(y_true, y_pred) :
    
    def f (y_true, y_pred) :
        
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    
    return tf.numpy_function(f, [y_true , y_pred] , tf.float32)
        
      

In [17]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint , ReduceLROnPlateau , CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall , Precision
from keras import optimizers
from tensorflow.keras.losses import BinaryFocalCrossentropy
  
        
if __name__ == "__main__":
    
    
    np.random.seed(42)
    tf.random.set_seed(42)
    batch = 16
    lr = 1e-1
    epochs = 100
    
    train_dataset= tf_dataset(train_x , train_y, batch = batch)
    valid_dataset= tf_dataset(valid_x , valid_y, batch = batch)
    
    
    model = build_model()
    
    binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()
    opt = tf.keras.optimizers.Adam(lr)
    
    metrics = ["acc" , Recall() , Precision() , iou , dice_coef]
    
    
    model.compile(loss = dice_coef_loss , optimizer = opt , metrics = metrics)
    
    callbacks  = [
        ModelCheckpoint("/kaggle/working/model.h5"),
        ReduceLROnPlateau(monitor = "val_loss" , factor = 0.8 , patience = 5),
        EarlyStopping(monitor = "val_loss" , patience = 30 , restore_best_weights = False)
    ]
    
    
    train_steps = len(train_x) // batch
    valid_steps = len(valid_x) // batch
    
    if len(train_x) % batch != 0 :
        train_steps+= 1
        
    if len(valid_x) % batch != 0 :
        valid_steps+= 1


    model.fit(
        
        train_dataset, 
        validation_data = valid_dataset,
        epochs = epochs,
        steps_per_epoch = train_steps,
        validation_steps = valid_steps,
        callbacks = callbacks,
        shuffle = False
    )

Epoch 1/100
31/31 [==============================] - 40s 590ms/step - loss: 0.7030 - acc: 0.7288 - recall: 0.6276 - precision: 0.2046 - iou: 0.1774 - dice_coef: 0.2975 - val_loss: 0.8438 - val_acc: 0.0826 - val_recall: 1.0000 - val_precision: 0.0876 - val_iou: 0.0847 - val_dice_coef: 0.1560 - lr: 0.1000
Epoch 2/100
31/31 [==============================] - 7s 219ms/step - loss: 0.6105 - acc: 0.8268 - recall: 0.5726 - precision: 0.2957 - iou: 0.2452 - dice_coef: 0.3902 - val_loss: 0.8438 - val_acc: 0.0826 - val_recall: 1.0000 - val_precision: 0.0876 - val_iou: 0.0847 - val_dice_coef: 0.1560 - lr: 0.1000
Epoch 3/100
31/31 [==============================] - 6s 200ms/step - loss: 0.5804 - acc: 0.8639 - recall: 0.5167 - precision: 0.3584 - iou: 0.2689 - dice_coef: 0.4203 - val_loss: 0.8438 - val_acc: 0.0826 - val_recall: 1.0000 - val_precision: 0.0876 - val_iou: 0.0847 - val_dice_coef: 0.1560 - lr: 0.1000
Epoch 4/100
31/31 [==============================] - 7s 224ms/step - loss: 0.5356 - acc

In [18]:
def read_image_new(path):
    
    x = cv2.imread(path , cv2.IMREAD_COLOR)
    x = cv2.resize(x , (256, 256))
    x = x/255.0
    # size is 256, 256 , 3
    return x


def read_mask_new(path):

    x = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x , (256, 256))
    # size is 256, 256
    x = np.expand_dims(x, axis = -1)
    # size is 256, 256 , 1
    
    return x

def mask_parse(mask):
    
    mask = np.squeeze(mask)
    mask = [mask , mask , mask]
    
    mask = np.transpose(mask , (1,2,0))
    return mask

In [19]:
from tqdm import tqdm
from tensorflow.keras.utils import CustomObjectScope



In [20]:
if __name__ == "__main__":
    
    test_dataset = tf_dataset(test_x , test_y , batch = batch)
    test_steps = len(test_x) // batch
    
    if len(test_x)%batch != 0 :
        test_steps +=1
        
    with CustomObjectScope({'iou' : iou , 'symmetric_focal_loss': symmetric_focal_loss, 'dice_coef' : dice_coef ,  'dice_coef_loss' :dice_coef_loss , 'tversky_loss':tversky_loss }):
    


        model = tf.keras.models.load_model("/kaggle/working/model.h5")
        
    model.evaluate(test_dataset , steps = test_steps)
    
    for i , (x,y) in tqdm(enumerate(zip(test_x , test_y)) , total= len(test_x)):
        
        x = read_image_new(x)
        y = read_mask_new(y)
        
        y_pred = model.predict(np.expand_dims(x , axis = 0))
        
        y_pred= y_pred[0] > 0.5
        
        h,w, _ = x.shape
        
        white_line = np.ones((h,10,3)) * 255.0
        
        all_images = [
            
            x*255.0, white_line,
            
            mask_parse(y) , white_line,
            mask_parse(y_pred)*255.0
            
        ]
        
        image = np.concatenate (all_images, axis =1)
        cv2.imwrite(f"/kaggle/working/{i}.png" , image)


NameError: name 'symmetric_focal_loss' is not defined

In [ ]:
!zip -r cvc_newnewnew.zip   "/kaggle/working/"

In [ ]:
y_pred.shape

In [ ]:
# !zip -r segmentation_model.zip  "/kaggle/input/segmentation-model"